# Parameter exploration for TCR/TCR antagonism in the classical model
The goal is to visualize why the François et al., 2013 model works badly on TCR/TCR antagonism data, and to ensure this does not stem from coding mistakes by verifying known outputs from that 2013 paper. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Local modules
import sys, os
if not "../" in sys.path:
    sys.path.insert(1, "../")

from mcmc.costs_tcr_tcr_antagonism import cost_antagonism_shp1
from mcmc.costs_tcr_tcr_antagonism import (antag_ratio_panel_shp1, steady_shp1_1ligand)
from mcmc.mcmc_analysis import find_best_grid_point, string_to_tuple
from mcmc.utilities_tcr_tcr_antagonism import (
    prepare_data, 
    check_fit_model_antagonism, 
    check_model_output,
    load_tcr_tcr_molec_numbers, 
    plot_fit_antagonism
)
from models.shp1_model import steady_shp1_1ligand, steady_shp1_2ligands

In [ ]:
# Import data
# Number of TCR per T cell, L-pulse conversion parameters, peptide taus
molec_counts_fi = os.path.join("..", "data", "surface_counts", "surface_molecule_summary_stats.h5")
mtc = "Geometric mean"
nums = load_tcr_tcr_molec_numbers(molec_counts_fi, mtc,
                                    tcell_type="OT1_Naive")
tcr_number, l_conc_mm_params, pep_tau_map_ot1 = nums

## Antagonism ratio fitting
# Prepare data for fitting antagonism ratios
data_file_name = os.path.join("..", "data", "antagonism", "allManualSameCellAntagonismDfs_v3.h5")
df = pd.read_hdf(data_file_name)
df_data, df_err, tau_agonist = prepare_data(df, l_conc_mm_params, pep_tau_map_ot1)

In [ ]:
# Set other parameters
# Model parameters
# Define the parameters that will remain the same throughout
# Parameters related to the cascade of phosphorylations
phi = 0.09
kappa = 1e-4
b = 0.04
gamma = 1.2e-6
N = 5
R_tot = tcr_number
I_tot = 6e5

# Bounds on parameters: phi, cmthresh, I_tot
# Use relatively tight bounds based on previous runs.
# Use the log of parameters so the MCMC steps are even in log scale
fit_bounds = [(0.05, 5.0), (1, 10*R_tot), (0.001*I_tot, 100*I_tot)]
fit_bounds = [np.log10(np.asarray(a)) for a in zip(*fit_bounds)]

# Wrapping up parameters
rates_others = [b, gamma, kappa]  # k_S will be gridded over

In [ ]:
# Import analysis results
with open (os.path.join("..", "results", "mcmc", "mcmc_analysis_shp1.json"), "r") as h:
    francois2013_analysis = json.load(h)

In [ ]:
best_m, best_pvec, best_cost = find_best_grid_point(francois2013_analysis, strat="best")
m_tuple = string_to_tuple(best_m)
nm_best = [N, m_tuple[0]]
print("Best kmf:", best_m)
print("Best fit:", 10.0**best_pvec)
print("Cost:", best_cost)

In [ ]:
from mcmc.plotting import data_model_handles_legend

In [ ]:
def change_log_ticks(axis, base=2, which="y"):
    if which == "y" or which == "both":
        ogyticks = axis.get_yticks()
        newyticks = list(np.unique([int(x) for x in ogyticks]))
        newyticklabels = [str(base)+'$^{'+str(x)+'}$' for x in newyticks]
        axis.set_yticks(newyticks)
        axis.set_yticklabels(newyticklabels)
    if which == "x" or which == "both":
        ogxticks = axis.get_xticks()
        newxticks = list(np.unique([int(x) for x in ogxticks]))
        newxticklabels = [str(base)+'$^{'+str(x)+'}$' for x in newxticks]
        axis.set_xticks(newxticks)
        axis.set_xticklabels(newxticklabels)
    return axis
    
def plot_fit_cost(pvec, bounds, m_fit, other_rates, rtot, n_p, tau_ag, df_ratio, df_ci, mm_params, 
                 panel_fct=antag_ratio_panel_shp1, cost_fct=cost_antagonism_shp1):
    # First, compute a model panel
    other_args = [other_rates, rtot, n_p, tau_ag]
    df_mod = check_fit_model_antagonism(panel_fct, pvec, m_fit, 
                        df_ratio, df_ci, other_args=other_args, n_taus=101)
    
    # Compute the cost function too and print it
    cost_val = cost_fct(pvec, bounds, m_fit, other_rates, rtot, n_p, tau_ag, df_ratio, df_ci)
    print("Cost function:", cost_val)
    
    # Make a nice plot 
    fig, ax = plot_fit_antagonism(df_ratio, df_mod, mm_params, df_ci, cost=cost_val, model_ci=None)
    
    return fig, ax

In [ ]:
def plot_model_output(expvec, other_rates, rtot, m_fit, n_p):
    model_rates = [expvec[0]] + list(other_rates) + [expvec[1]]
    ritot = [rtot, expvec[2]]
    res = check_model_output(steady_shp1_1ligand, model_rates, ritot, [n_p, m_fit[0]])

    l_range, tau_range, outputs = res
    fig, ax = plt.subplots()
    fig.set_size_inches(5.5, 4.0)
    for i, tau in enumerate(tau_range):
        ax.plot(l_range, outputs[i], label=r"$\tau = {:.0f}$ s".format(tau))
    ax.set(xscale="log", yscale="log", xlabel=r"$L$", ylabel=r"$C_N$", title="AKPR SHP-1 model")
    ax.legend(loc="lower right")
    ax.annotate(r"Best $m$ : ${}$".format(m_fit[0]) + "\n"
                + r"Best $\phi$ : " + "{:.2f}\n".format(expvec[0])
                + r"Best $C_{m,thresh}$ : " + "{:.2e}\n".format(expvec[1])
                + r"Best $S_{tot}$ :" + "{:.2e}".format(expvec[2]),
                xy=(0.05, 0.95), ha="left", va="top",
                xycoords="axes fraction")
    fig.tight_layout()

    plt.show()
    plt.close()
    return l_range, tau_range, outputs

In [ ]:
# Check the best fit from the simulations first
df_model, cost_test = plot_fit_cost(best_pvec, fit_bounds, m_tuple, rates_others, R_tot, 
                                     N, tau_agonist, df_data, df_err, l_conc_mm_params)

_ = plot_model_output(10**best_pvec, rates_others, R_tot, m_tuple, N)

What the hell is happening with this model?

# Try manually tweaking the best fit
To capture better the 1 nM antagonist, 10 pM agonist condition. 

In [ ]:
# Try kind of default parameters, see how they predict antagonism
tweak_pvec = np.log10([0.1, 500, 6e5])

df_model, cost_test = plot_fit_cost(tweak_pvec, fit_bounds, [1], rates_others, R_tot, 
                                     N, tau_agonist, df_data, df_err, l_conc_mm_params)

_ = plot_model_output(10**tweak_pvec, rates_others, R_tot, [1], N)

## Reproduce a figure from François *et al*., 2013 as a final check

In [ ]:
# Reproduce a figure from François 2013 as a final check
l1_axis = np.logspace(0, 4, 101)
tau2_axis = [0.5, 1.0, 1.5, 2.0, 2.5]
l2_fixed = 1e4
tau1_fixed = 10.0
original_rates = [0.09, 0.04, 1.2e-6, 1e-4, 500.0]
ritots_original = [3e4, 6e5]
nmp_original = [5, 1]

# Even reusing the colors of the original PNAS 2013 figure 3C
colors = plt.cm.jet(np.linspace(0.0, 1.0, 9))
colors = colors[1:len(tau2_axis)+2]

antagonism_curves = np.zeros([len(tau2_axis)+1, len(l1_axis)])
for j in range(len(l1_axis)):
    # No antagonist curve
    antagonism_curves[0, j] = steady_shp1_1ligand(original_rates, tau1_fixed, 
                                        l1_axis[j], ritots_original, nmp_original)[nmp_original[0]]
for i in range(1, antagonism_curves.shape[0]):
    for j in range(antagonism_curves.shape[1]):
        complexes = steady_shp1_2ligands(original_rates, tau1_fixed, tau2_axis[i-1],
                             l1_axis[j], l2_fixed, ritots_original, nmp_original)
        antagonism_curves[i, j] = complexes[nmp_original[0]] + complexes[2*nmp_original[0]+1]

fig, ax = plt.subplots()
ax.axhline(0.2, ls=":", color="grey")
ax.plot(l1_axis, antagonism_curves[0], label="No antagonist", color=colors[0])
for i in range(1, antagonism_curves.shape[0]):
    ax.plot(l1_axis, antagonism_curves[i], label=r"$\tau = {:.1f}$ s".format(tau2_axis[i-1]), 
           color=colors[i])
ax.set(xscale="log", yscale="log", xlabel="L", ylabel=r"$C_N + D_N$")

ax.legend()
plt.show()
plt.close()

So there does not seem to be anything wrong in the code. The model just cannot capture well the real shape of antagonism (at the level of cytokines, at least).  